# Yandex messenger cleaner

В компании для общения используем Яндекс.Телемост и Яндекс.Мессенджер.

Практически любой созвон в Яндекс.Телемост создает отдельный чат в Яндекс.Мессенджер, со временем их накапливается очень много и начинает все тормозить.

Удобство и красоту обоих продуктов я оставлю за рамками ;

В данном ноутбуке описана методика удаления чатов созданных во время встреч - они часто пыстые или содержат временную информацию

__ВНИМАНИЕ! Перед удалением - убедитесь, что вы сохранили необходимую информацию из чатов__

## Поиск списка чатов

1. открываем в Chrome Яндекс.Мессенджер
2. открываем Инструменты разработчика, выбираем вкладку Network
3. выставляем галки `Preserve log`, `Disable cache`
4. перезагружаем страницу с Яндекс.Мессенджер
5. в списке запросов ищем что то типа: `websocket?....`, если гнянуть на URL: `wss://push.yandex.ru/v2/subscribe/websocket?service=messenger-prod%3Aversion5*common%2Bversion5*main&session=....`
6. видим обмен сообщениями по websocket
    -  нам нужно повторяющееся самое большое по размеру сообщение, у меня около 260кб
    - тыкаем в него правой кнопкой и копируем как UTF-8 и вставляем в VSCode
    - удаляем бинарный заголовок, сохраняем как Json
    


In [15]:
import json
import requests

In [9]:
f = open('chats-1.json')
json_data = json.load(f)

In [10]:
json_data.keys()

dict_keys(['Chats', 'RequestId', 'UnreadCount'])

In [12]:
len(json_data['Chats'])

695

695 чатов - охренеть!!!

In [ ]:
# выведим первые три
json_data['Chats'][:3]

## Выделяем запрос на удаление чата

- Идем обрабнов в браузер, Инструменыт разработчика, можно очистить все что там налетело
- Переключаемся на вкладку с Я.Мессенджер, выбираем чат который хотим удалить
- Еще раз чистим историю в Инструментах разработчика, вкладка `Network`
- Жмем правой кнопкой на чате и выбираем `Покинуть чат`
- Ищем в `Neteork` запрос к `/api/`
- Копируем как `cUrl`, получится что-то типа

```bash
curl 'https://yandex.ru/messenger/api/registry/api/' \
  -H 'authority: yandex.ru' \
  -H 'accept: application/json' \
  -H 'accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' \
  ....
  -H 'x-version: 5' \
  -H 'x-ya-organization-id: 0' \
  --data-raw $'------WebKitFormBoundaryOXyaKLjlkasd3\r\nContent-Disposition: form-data; name="request"\r\n\r\n{"method":"leave","params":{"chat_id":"0/22/f98dac44-abcb-4403-a74c-c48697321355","version":1679644323701088}}\r\n------WebKitFormBoundaryOXyaKLjlkasd3--\r\n' \
  --compressed
```
- теперь нужно научиться формировать такие запросы для не нужных нам чатов. При этом не обязательно повторять все заголовки, но часть из них важные:
    - authority: yandex.ru
    - accept: application/json
    - cookie: ...
    - origin: https://yandex.ru
    - referer: https://yandex.ru/chat
    - x-csrf-token: ...
    - x-request-id: 168027537453462328'

POST запрос c данными:

```json
{
    "method":"leave",
    "params":{
        "chat_id":"0/22/f98dac44-abcb-4403-a74c-123456789009",
        "version":1679644323701088
    }
}
```

In [28]:
# укажем параметры из заголовков
cookie = 'yuidss=....'
csrf_token = '....'

In [ ]:
import requests

url = 'https://yandex.ru/messenger/api/registry/api/'
headers = {'authority': 'yandex.ru',
           'accept': 'application/json',
           'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
           'cookie': cookie,
           'x-csrf-token': csrf_token
          }

jdata = {
    "method":"leave",
    "params":{
        "chat_id":"0/22/f98dac44-abcb-4403-a74c-123456789009",
        "version":16796443453701088
    }
}

x = requests.post(url, headers=headers, json = jdata)

print(x.text)

Опытным путем вычислил, что chat_id которые создаются Телемостом, начинаются с `0/22/`, **возможно у вас подругому**

In [43]:
# посчитаем сколько у нас чатов со встреч
cnt = 0
for chat in json_data['Chats']:
    chat_id = chat['ChatId']
    counters = chat['Counters']
    if chat_id.startswith('0/22/'):
        cnt += 1
print(cnt)

483


In [44]:
483 / 695

0.6949640287769784

Отфильтруем их и удалим

In [ ]:
for chat in json_data['Chats']:
    chat_id = chat['ChatId']
    counters = chat['Counters']
    if chat_id.startswith('0/22/'):
        print(f'chat_id: {chat_id}, counters: {counters}')
        jdata = {
            "method":"leave",
            "params":{
                "chat_id": chat_id,
                "version":1679644523701088
            }
        }        
        x = requests.post(url, headers=headers, json = jdata)
        print(x.text)